In [56]:
import numpy as np
import pandas as pd
from glob import glob

In [57]:
output = []
for path in glob("2021/*.xlsx"):
    print(path)
    xlsx = pd.ExcelFile(path)

    for sheet in xlsx.sheet_names:
        frag = pd.read_excel(xlsx, sheet_name=sheet)
        output.append(frag)

df = pd.concat(output, ignore_index=True)
df.head()

2021\01_Resultados_Mesa_Concejales_TER.xlsx
2021\02_Resultados_Mesa_Concejales_TER.xlsx
2021\03_Resultados_Mesa_Concejales_TER.xlsx
2021\04_Resultados_Mesa_Concejales_TER.xlsx
2021\05_Resultados_Mesa_Concejales_TER.xlsx
2021\06_Resultados_Mesa_Concejales_TER.xlsx
2021\07_Resultados_Mesa_Concejales_TER.xlsx
2021\08_Resultados_Mesa_Concejales_TER.xlsx
2021\09_Resultados_Mesa_Concejales_TER.xlsx
2021\10_Resultados_Mesa_Concejales_TER.xlsx
2021\11_Resultados_Mesa_Concejales_TER.xlsx
2021\12_Resultados_Mesa_Concejales_TER.xlsx
2021\13_Resultados_Mesa_Concejales_TER.xlsx
2021\14_Resultados_Mesa_Concejales_TER.xlsx
2021\15_Resultados_Mesa_Concejales_TER.xlsx
2021\16_Resultados_Mesa_Concejales_TER.xlsx


,Región Nº,Región,Provincia,CS,Distrito,Comuna,Circunscripción,Mesa Nº,Tipo,Fusionadas,...,Sub Pacto,Partido,Candidato,Votos TER,N° Región,Nº Mesa,SubPacto,N° mesa,Subpacto,MesaNº
0,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,KARL HEINZ HARDER LLANOS,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,HUMBERTO SADY GALAZ ROJAS,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GIGLIOLA ANGELICA INOSTROZA ESCARATE,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,ESTER YAZMIN CISTERNAS VERA,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GISELLE DEL CARMEN MUÑOZ GONZALEZ,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df = pd.concat(output, ignore_index=True)
df.head()

,Región Nº,Región,Provincia,CS,Distrito,Comuna,Circunscripción,Mesa Nº,Tipo,Fusionadas,...,Sub Pacto,Partido,Candidato,Votos TER,N° Región,Nº Mesa,SubPacto,N° mesa,Subpacto,MesaNº
0,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,KARL HEINZ HARDER LLANOS,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,HUMBERTO SADY GALAZ ROJAS,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GIGLIOLA ANGELICA INOSTROZA ESCARATE,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,ESTER YAZMIN CISTERNAS VERA,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GISELLE DEL CARMEN MUÑOZ GONZALEZ,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df["Votos TER"] = df["Votos TER"].astype(str).str.strip().replace("", 0).astype(float)


In [17]:
df[df.Candidato.fillna("#").str.contains("NULO")].Candidato.unique()

array(['VOTOS NULOS'], dtype=object)

In [62]:
def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

df["polling_id"] = create_polling_id(df, columns=["Región", "Provincia", "Distrito", "Comuna", "Circunscripción", "Mesa Nº", "Tipo", "Fusionadas"])

In [63]:
# VOTOS NULOS
# VOTOS EN BLANCO	
# TOTALES
df["Candidato"] = df["Candidato"].replace({
    "VOTOS EN BLANCO": "BLANK",
    "VOTOS EN BLANCOS": "BLANK",
    "VOTOS BLANCOS": "BLANK",
    "VOTOS NULOS": "NULL"
})

df_spoilt = df[df["Candidato"].isin(["NULL", "BLANK"])].copy()
df_spoilt = df_spoilt[["Candidato", "Votos TER", "polling_id"]].rename(columns={"Candidato": "candidate", "Votos TER": "value"})
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

df = df[~df["Candidato"].isin(["BLANK", "NULL", "TOTALES", np.nan])].copy()


In [64]:
data = df[["polling_id", "Pacto", "Votos TER"]].dropna(subset=["Pacto"])
data = data.rename(columns={"Pacto": "candidate", "Votos TER": "value"})
data["value"] = data["value"].astype(str).apply(lambda x: x.strip()).apply(lambda x: 0 if x == "" else x).astype(float)

tt = data.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt["flag_candidates"] = 1

tt.head()

,polling_id,candidate,value,rate,flag_candidates
0,ARICA Y PARINACOTA-ARICA-1.0-ARICA-ARICA NORTE...,CHILE DIGNO VERDE Y SOBERANO,22.0,0.189655,1
1,ARICA Y PARINACOTA-ARICA-1.0-ARICA-ARICA NORTE...,CHILE VAMOS - PRI E INDEPENDIENTES,4.0,0.034483,1
2,ARICA Y PARINACOTA-ARICA-1.0-ARICA-ARICA NORTE...,CHILE VAMOS EVOPOLI - INDEPENDIENTES,5.0,0.043103,1
3,ARICA Y PARINACOTA-ARICA-1.0-ARICA-ARICA NORTE...,CHILE VAMOS RENOVACION NACIONAL - INDEPENDIENTES,24.0,0.206897,1
4,ARICA Y PARINACOTA-ARICA-1.0-ARICA-ARICA NORTE...,CHILE VAMOS UDI - INDEPENDIENTES,8.0,0.068966,1


In [65]:
data = pd.concat([
    tt,
    df_spoilt
])

In [66]:
df_location = df[["polling_id", "Región", "Provincia", "Distrito", "Comuna", "Circunscripción", "Mesa Nº", "Tipo"]].drop_duplicates()
df_location = df_location.rename(columns={
    "Región": "region",
    "Comuna": "commune",
    "Provincia": "province",
    "Distrito": "district",
    "Circunscripción": "circunscription",
    "Mesa Nº": "polling_station",
    "Tipo": "type"
})

In [67]:
df_location.head()

,polling_id,region,province,district,commune,circunscription,polling_station,type
0,DE TARAPACA-IQUIQUE-2.0-ALTO HOSPICIO-ALTO HOS...,DE TARAPACA,IQUIQUE,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M
42,DE TARAPACA-IQUIQUE-2.0-ALTO HOSPICIO-ALTO HOS...,DE TARAPACA,IQUIQUE,2.0,ALTO HOSPICIO,ALTO HOSPICIO,2.0,M
84,DE TARAPACA-IQUIQUE-2.0-ALTO HOSPICIO-ALTO HOS...,DE TARAPACA,IQUIQUE,2.0,ALTO HOSPICIO,ALTO HOSPICIO,3.0,M
126,DE TARAPACA-IQUIQUE-2.0-ALTO HOSPICIO-ALTO HOS...,DE TARAPACA,IQUIQUE,2.0,ALTO HOSPICIO,ALTO HOSPICIO,4.0,M
168,DE TARAPACA-IQUIQUE-2.0-ALTO HOSPICIO-ALTO HOS...,DE TARAPACA,IQUIQUE,2.0,ALTO HOSPICIO,ALTO HOSPICIO,5.0,M


In [69]:
data.to_csv("../data_output/Chile/2021_councilor.csv.gz")
df_location.to_csv("../data_output/Chile/2021_councilor_location.csv.gz")